In [54]:
import pickle 
import torch

In [57]:
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"

# Loading the attention file
batch1_attentions = pickle.load(open(dir + "shanghai_batch1_finetuneAttention.pkl", "rb"))
batch2_attentions = pickle.load(open(dir + "shanghai_batch2_finetuneAttention.pkl", "rb"))
batch3_attentions = pickle.load(open(dir + "shanghai_batch3_finetuneAttention.pkl", "rb"))
batch4_attentions = pickle.load(open(dir + "shanghai_batch4_finetuneAttention.pkl", "rb"))
batch5_attentions = pickle.load(open(dir + "shanghai_batch5_finetuneAttention.pkl", "rb"))
batch6_attentions = pickle.load(open(dir + "shanghai_batch6_finetuneAttention.pkl", "rb"))

# colas_attentions = pickle.load(open(dir + "Colas_1028_attention.pkl", "rb"))

# batch4_attentions = pickle.load(file)
# torch.load(file, map_location=torch.device('cpu'))

# Stacked into tensor
batch1_attentions = torch.stack(batch1_attentions, dim=0)
batch2_attentions = torch.stack(batch2_attentions, dim=0)
batch3_attentions = torch.stack(batch3_attentions, dim=0)
batch4_attentions = torch.stack(batch4_attentions, dim=0)
batch5_attentions = torch.stack(batch5_attentions, dim=0)
batch6_attentions = torch.stack(batch6_attentions, dim=0)

# colas_attentions = torch.stack(colas_attentions, dim=0)

print(batch1_attentions.shape)
print(batch2_attentions.shape)
print(batch3_attentions.shape)
print(batch4_attentions.shape)
print(batch5_attentions.shape)
print(batch6_attentions.shape)
# print(colas_attentions.shape)

torch.Size([4, 300, 8, 289, 289])
torch.Size([4, 300, 8, 289, 289])
torch.Size([4, 300, 8, 289, 289])
torch.Size([4, 300, 8, 289, 289])
torch.Size([4, 300, 8, 289, 289])
torch.Size([4, 417, 8, 289, 289])


In [58]:
final_merged_attentions = torch.cat([batch1_attentions, batch2_attentions, batch3_attentions, batch4_attentions, batch5_attentions, batch6_attentions], dim=1)
print(final_merged_attentions.shape)

torch.Size([4, 1917, 8, 289, 289])


In [46]:
output_file_path = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/shanghai_total_attention.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(final_merged_attentions, f)

In [59]:

averaged_attentions = torch.mean(final_merged_attentions, dim=3)
# averaged_attentions = torch.mean(colas_attentions, dim=3)

print(averaged_attentions.shape)  #  (4, 1917, 8, 289)

torch.Size([4, 1917, 8, 289])


In [5]:
output_file_path = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/colas_1028_attention_meanDim3.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(averaged_attentions, f)

In [60]:
output_file_path = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/shanghai_total_finetuneAttention_meanDim3.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(averaged_attentions, f)

In [112]:
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'shanghai_total_attention_meanDim3.pkl', 'rb') as f:
    attention_averages = pickle.load(f)

In [113]:
attention_averages.shape

torch.Size([4, 1917, 8, 289])

In [95]:
first_position = attention_averages[:, :, :, :1].squeeze(-1)
# remaining_positions = attention_averages[:, :, :, 1:].mean(dim=-1, keepdim=True)
remaining_positions = attention_averages[:, :, :, 1:].mean(dim=-1)

In [96]:
print(first_position.shape)
print(remaining_positions.shape)

torch.Size([4, 1917, 8])
torch.Size([4, 1917, 8])


In [97]:
print(first_position.dtype)
print(remaining_positions.dtype)

torch.float32
torch.float32


In [110]:
first_position[0].shape

torch.Size([1917, 8])

In [108]:
output_file_path = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/shanghai_total_attention_CLS.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(first_position, f)

In [27]:
from datasets import load_from_disk
# dataset_path = "/share/home/liangzhongming/930/CGMformer/data/8_11_data/Shanghai_total"
dataset_path ="/share/home/liangzhongming/930/CGMformer/data/Data_downstream/Colas_1028_hfds"
dataset = load_from_disk(dataset_path)



In [28]:
dataset

Dataset({
    features: ['input_ids', 'types', 'index', 'id', 'p_id', 'hba1c', 'filled'],
    num_rows: 393
})

In [29]:
import pandas as pd
labels_df = pd.read_csv("/share/home/liangzhongming/930/CGMformer/data/8_11_data/CGM/For_LZM_1028/Colas_DataVar.csv", index_col=0)

In [30]:
averaged_attentions.shape

torch.Size([4, 393, 8, 289])

In [12]:
attention_averages = []

for sample_index in range(1917): # 1917 393
    # sample_attention = averaged_attentions[:, sample_index, :, 1:]
    sample_attention = attention_averages[:, sample_index, :, 1:]

    # print(dataset['input_ids'][sample_index])
    
    input_ids = torch.tensor(dataset['input_ids'][sample_index])
    positions = torch.where(input_ids < 70)
        
    
    # print(positions)
    sample_attentions_list = []

    print(positions[0])
    if len(positions[0]) == 0:
        
        sample_attentions = torch.zeros(sample_attention.shape)
    else:
        for pos in positions[0]:
            sub_attention = sample_attention[:, :, pos]
            sample_attentions_list.append(sub_attention)
    
        sample_attentions = torch.stack(sample_attentions_list, dim=2)
        
    average_attention = sample_attentions.mean(dim=-1)
#     print(f"average_attention : {average_attention}")

    attention_averages.append(average_attention)
#     if sample_index == 10:
#         break

tensor([222])
tensor([], dtype=torch.int64)
tensor([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76])
tensor([24, 25, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78,
        79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])
tensor([50, 51, 52, 53, 54, 55, 56, 94, 95, 96, 97])
tensor([34, 35, 36, 37, 38, 39, 40, 41, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96,
        97, 98])
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117])
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([71, 72, 73, 87, 88, 89, 90, 91, 92])
tensor([], dtype=torch.int64)
tensor([], dt

tensor([], dtype=torch.int64)
tensor([51, 52, 53, 54, 55, 56, 57])
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  80,  81,  82,  83,  84,  85,  86, 194, 195, 196, 197, 198, 199,
        200, 201])
tensor([170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
        184, 185, 186, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,
        230, 231, 232, 233, 234, 235, 243, 244, 245, 246, 247, 248, 249, 250,
        251, 252, 253, 254, 255, 256, 257, 269, 270, 271, 272, 273, 274, 275,
        276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287])
tensor([], dtype=torch.int64)
tensor([147, 148, 149, 150])
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([ 55,  56, 

tensor([ 19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
        182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192])
tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([], dtype=t

tensor([], dtype=torch.int64)
tensor([], dtype=torch.int64)
tensor([ 43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  63, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257,
        258, 259, 260, 261, 262, 263, 264])
tensor([ 31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
         59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
         73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84, 151, 152,
        153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 168, 169, 170, 171, 172, 173, 212, 213, 214, 215, 216, 217, 218,
        219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
        233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246,
        247, 248, 249, 250, 251, 252, 

In [13]:
print(len(attention_averages))
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'Colas_low_glu_avgAttentions.pkl', 'wb') as f:
    pickle.dump(attention_averages, f)

393


In [19]:
N_attention_averages = []
Up_attention_averages = []
Down_attention_averages = []

for sample_index in range(1917):
    sample_attention = averaged_attentions[:, sample_index, :, 1:] # 排除cls
    
    labels = labels_df.loc[sample_index]
    # print(labels)
    
    n_positions = []
    up_positions = []
    down_positions = []
    for i in range(288):
        label = labels[i]

        if label == "N":
            n_positions.append(i)
        elif label == "Up":
            up_positions.append(i)
        elif label == "Down":
            down_positions.append(i)

    # print(positions)
    attentions_list_n = []
    attentions_list_up = []
    attentions_list_down = []

    print(n_positions)
    if len(n_positions) == 0:
        sample_attentions_n = torch.zeros(sample_attention.shape)
    else:
        for pos in n_positions:
            sub_attention = sample_attention[:, :, pos]
            attentions_list_n.append(sub_attention)
        sample_attentions_n = torch.stack(attentions_list_n, dim=2)
        
    average_attention_n = sample_attentions_n.mean(dim=-1)

    N_attention_averages.append(average_attention_n)
    # Up_attention_averages.append(average_attention_n)
    # Down_attention_averages.append(average_attention_n)


[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 24, 25, 26, 33, 34, 47, 48, 52, 53, 54, 62, 63, 69, 70, 77, 78, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 111, 112, 113, 114, 115, 125, 126, 134, 135, 136, 140, 141, 160, 161, 162, 169, 170, 189, 190, 196, 197, 205, 206, 223, 224, 233, 234, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 266, 267, 268, 282, 283, 284, 285, 286, 287]
[0, 1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 23, 24, 37, 38, 39, 45, 46, 60, 61, 65, 66, 67, 72, 73, 74, 106, 107, 108, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 157, 158, 166, 167, 195, 196, 197, 229, 230, 231, 239, 240, 241, 242, 262, 263, 264, 265, 275, 276, 282, 283, 284, 285, 286, 287]
[0, 1, 2, 3, 4, 5, 22, 23, 24, 27, 28, 29, 34, 35, 36, 42, 43, 51, 52, 60, 61, 62, 70, 71, 82, 83, 84, 85, 86, 100, 101, 102, 103, 107, 108, 122, 123, 154, 155, 156, 161, 162, 163, 167, 168, 169, 180, 181, 182, 184, 185, 

[0, 1, 2, 3, 4, 5, 16, 17, 18, 19, 20, 21, 30, 31, 32, 41, 42, 43, 57, 58, 59, 62, 63, 64, 65, 67, 68, 69, 70, 71, 83, 84, 88, 89, 90, 101, 102, 105, 106, 107, 129, 130, 172, 173, 174, 199, 200, 212, 213, 214, 230, 231, 232, 244, 245, 246, 247, 248, 249, 258, 259, 260, 267, 268, 282, 283, 284, 285, 286, 287]
[0, 1, 2, 3, 4, 5, 11, 12, 32, 33, 34, 46, 47, 48, 52, 65, 66, 77, 78, 79, 89, 90, 101, 102, 113, 114, 115, 117, 118, 130, 131, 137, 138, 139, 142, 143, 144, 176, 177, 178, 189, 190, 200, 201, 202, 208, 209, 210, 214, 215, 221, 222, 223, 231, 232, 233, 242, 243, 253, 254, 262, 263, 264, 273, 274, 275, 282, 283, 284, 285, 286, 287]
[0, 1, 2, 3, 4, 5, 6, 13, 14, 15, 40, 41, 42, 50, 51, 52, 56, 57, 58, 59, 63, 64, 65, 87, 88, 99, 100, 106, 107, 108, 113, 114, 115, 119, 120, 129, 130, 137, 138, 149, 150, 159, 160, 161, 164, 165, 166, 180, 181, 191, 192, 203, 204, 215, 216, 221, 222, 223, 231, 232, 233, 237, 238, 239, 244, 245, 263, 264, 265, 282, 283, 284, 285, 286, 287]
[0, 1, 2, 3, 4

IndexError: index 393 is out of bounds for dimension 1 with size 393

In [20]:
len(Down_attention_averages)
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'Colas_N_avgAttentions.pkl', 'wb') as f:
    pickle.dump(N_attention_averages, f)

In [41]:
attention_averages = []
total_position = list(range(288))
positions = total_position[216:288]

for sample_index in range(393):
    sample_attention = averaged_attentions[:, sample_index, :, 1:]
    
    # print(dataset['input_ids'][sample_index])
    
    input_ids = torch.tensor(dataset['input_ids'][sample_index])
    # positions = torch.where(input_ids > 180)
    
    # print(positions)
    sample_attentions_list = []

    print(positions)
    if len(positions) == 0:
        sample_attentions = torch.zeros(sample_attention.shape)
    else:
        for pos in positions:
            sub_attention = sample_attention[:, :, pos]
            sample_attentions_list.append(sub_attention)
        sample_attentions = torch.stack(sample_attentions_list, dim=2)
        

    average_attention = sample_attentions.mean(dim=-1)
    # print(f"average_attention : {average_attention}")

    attention_averages.append(average_attention)
#     if sample_index == 10:
#         break

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

...

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 27

In [42]:
print(len(attention_averages))
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'Colas_216_288_avgAttentions.pkl', 'wb') as f:
    pickle.dump(attention_averages, f)

393


In [45]:
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/Colas_attentions/"
with open(dir + 'Colas_216_288_avgAttentions.pkl', 'rb') as f:
    attention_averages = pickle.load(f)

In [53]:
attention_averages

[tensor([[0.0028, 0.0024, 0.0035, 0.0015, 0.0023, 0.0042, 0.0026, 0.0025],
         [0.0036, 0.0023, 0.0051, 0.0032, 0.0024, 0.0025, 0.0030, 0.0028],
         [0.0019, 0.0022, 0.0015, 0.0020, 0.0018, 0.0030, 0.0011, 0.0016],
         [0.0032, 0.0044, 0.0026, 0.0023, 0.0030, 0.0032, 0.0020, 0.0039]]),
 tensor([[0.0032, 0.0037, 0.0031, 0.0028, 0.0038, 0.0037, 0.0037, 0.0037],
         [0.0031, 0.0028, 0.0033, 0.0028, 0.0029, 0.0027, 0.0033, 0.0034],
         [0.0021, 0.0037, 0.0030, 0.0033, 0.0027, 0.0029, 0.0032, 0.0029],
         [0.0034, 0.0028, 0.0037, 0.0026, 0.0029, 0.0033, 0.0027, 0.0037]]),
 tensor([[0.0034, 0.0040, 0.0028, 0.0014, 0.0037, 0.0045, 0.0037, 0.0036],
         [0.0040, 0.0032, 0.0023, 0.0038, 0.0041, 0.0040, 0.0034, 0.0031],
         [0.0052, 0.0042, 0.0023, 0.0031, 0.0047, 0.0027, 0.0033, 0.0031],
         [0.0028, 0.0046, 0.0038, 0.0034, 0.0024, 0.0022, 0.0033, 0.0032]]),
 tensor([[0.0033, 0.0027, 0.0030, 0.0026, 0.0026, 0.0035, 0.0028, 0.0027],
         [0.0037, 0

In [140]:
import pandas as pd
timeSplit_df = pd.read_csv("/share/home/liangzhongming/930/CGMformer/data/8_11_data/CGM/For_LZM_1028/Shanghai_TimeSplit.csv", index_col=0)

In [141]:
timeSplit_df

,Fast_s,Fast_e,Dawn_s,Dawn_e,Breakfast_s,Breakfast_e,Lunch_s,Lunch_e,Dinner_s,Dinner_e
0,0,72,48,72,NaN,NaN,NaN,NaN,238.0,262.0
1,0,72,48,72,NaN,NaN,NaN,NaN,227.0,251.0
2,0,95,48,95,95.0,119.0,140.0,164.0,238.0,262.0
3,0,111,48,111,111.0,135.0,152.0,176.0,232.0,256.0
4,0,72,48,72,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1912,0,88,48,88,88.0,112.0,149.0,173.0,223.0,247.0
1913,0,91,48,91,91.0,115.0,150.0,174.0,205.0,229.0
1914,0,98,48,98,98.0,122.0,143.0,167.0,221.0,245.0
1915,0,103,48,103,103.0,127.0,143.0,167.0,216.0,240.0


In [144]:
attention_averages = []

for sample_index in range(1917):
    sample_attention = averaged_attentions[:, sample_index, :, 1:]
    

    fast_s = timeSplit_df.loc[sample_index, 'Dawn_s'] 
    fast_e = timeSplit_df.loc[sample_index, 'Dawn_e']
    fast_positions = list(range(fast_s, fast_e + 1)) 
    
    sample_attentions_list = []

    print(fast_positions)
    if len(fast_positions) == 0:
        sample_attentions = torch.zeros(sample_attention.shape)
    else:
        for pos in fast_positions:
            sub_attention = sample_attention[:, :, pos]
            sample_attentions_list.append(sub_attention)
        sample_attentions = torch.stack(sample_attentions_list, dim=2)
        
    average_attention = sample_attentions.mean(dim=-1)
    # print(f"average_attention : {average_attention}")

    attention_averages.append(average_attention)
#     if sample_index == 10:
#         break

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72]
[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70

In [145]:
print(len(attention_averages))
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'shanghai_Dawn_avgAttentions.pkl', 'wb') as f:
    pickle.dump(attention_averages, f)

1917


In [154]:
import numpy as np
attention_averages = []

for sample_index in range(1917):
    sample_attention = averaged_attentions[:, sample_index, :, 1:]
    
    breakfast_s = timeSplit_df.loc[sample_index, 'Breakfast_s'] 
    lunch_s = timeSplit_df.loc[sample_index, 'Lunch_s'] 
    dinner_s = timeSplit_df.loc[sample_index, 'Dinner_s'] 
    
#     print(f"breakfast_s:{breakfast_s}")
#     print(f"lunch_s:{lunch_s}")
#     print(f"dinner_s:{dinner_s}")
    
#     print(f"breakfast_s isnan:{np.isnan(breakfast_s)}")
#     print(f"lunch_s isnan:{np.isnan(lunch_s)}")
#     print(f"dinner_s isnan:{np.isnan(dinner_s)}")
    
    beforeMeal_positions = []
      # 如果breakfast非空,添加其前18个位置
    if not np.isnan(breakfast_s):
        start = int(breakfast_s) - 18
        end = int(breakfast_s)
        beforeMeal_positions.extend(list(range(start, end))) # 左闭右开

    if not np.isnan(lunch_s):
        start = int(lunch_s) - 18  
        end = int(lunch_s)
        beforeMeal_positions.extend(list(range(start, end)))

    if not np.isnan(dinner_s):  
        start = int(dinner_s) - 18
        end = int(dinner_s)
        beforeMeal_positions.extend(list(range(start, end)))

#     if sample_index == 10:
#         break
    

    sample_attentions_list = []

    print(beforeMeal_positions)
    if len(beforeMeal_positions) == 0:
        sample_attentions = torch.zeros(sample_attention.shape)
    else:
        for pos in beforeMeal_positions:
            sub_attention = sample_attention[:, :, pos]
            sample_attentions_list.append(sub_attention)
        sample_attentions = torch.stack(sample_attentions_list, dim=2)
        
    average_attention = sample_attentions.mean(dim=-1)
    # print(f"average_attention : {average_attention}")

    attention_averages.append(average_attention)
#     if sample_index == 10:
#         break

[220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237]
[209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226]
[77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237]
[93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231]
[]
[]
[]
[]
[]
[136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233]
[76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 200, 201, 202, 203, 204, 205, 20

[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213]
[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214]
[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214

[122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219]
[76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223]
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213]
[69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,

In [155]:
print(len(attention_averages))
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'shanghai_beforeMeal_avgAttentions.pkl', 'wb') as f:
    pickle.dump(attention_averages, f)

1917


In [160]:
import numpy as np
attention_averages = []

for sample_index in range(1917):
    sample_attention = averaged_attentions[:, sample_index, :, 1:]

    breakfast_e = timeSplit_df.loc[sample_index, 'Breakfast_e']
    lunch_s = timeSplit_df.loc[sample_index, 'Lunch_s']
    lunch_e = timeSplit_df.loc[sample_index, 'Lunch_e'] 
    dinner_s = timeSplit_df.loc[sample_index, 'Dinner_s']
    dinner_e = timeSplit_df.loc[sample_index, 'Dinner_e']

    afterMeal_positions = []

    if not(np.isnan(breakfast_e) or np.isnan(lunch_s)):  
        start = int(breakfast_e) 
        end = int(lunch_s)
        afterMeal_positions.extend(list(range(start + 1, end))) # ()

    if not(np.isnan(lunch_e) or np.isnan(dinner_s)):
        start = int(lunch_e)
        end = int(dinner_s) 
        afterMeal_positions.extend(range(start + 1, end))
 
    if not np.isnan(dinner_e):
        start = int(dinner_e)
        end = 288
        afterMeal_positions.extend(range(start + 1, end))

#     if sample_index == 10:
#         break
    

    sample_attentions_list = []

    print(afterMeal_positions)
    if len(afterMeal_positions) == 0:
        sample_attentions = torch.zeros(sample_attention.shape)
    else:
        for pos in afterMeal_positions:
            sub_attention = sample_attention[:, :, pos]
            sample_attentions_list.append(sub_attention)
        sample_attentions = torch.stack(sample_attentions_list, dim=2)
        
    average_attention = sample_attentions.mean(dim=-1)
#     print(f"average_attention : {average_attention}")

    attention_averages.append(average_attention)
#     if sample_index == 10:
#         break

[263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 177, 178, 179, 180, 1

[112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[]
[113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 

[110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[]
[]
[113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]
[141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 2

[117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141]
[118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[]
[]
[116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 1

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[]
[]
[]
[125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 2

[117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]
[]
[]
[124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 25

In [161]:
print(len(attention_averages))
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'shanghai_afterMeal_avgAttentions.pkl', 'wb') as f:
    pickle.dump(attention_averages, f)

1917


In [156]:
dir = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/"
with open(dir + 'shanghai_beforeMeal_avgAttentions.pkl', 'rb') as f:
    attention_averages = pickle.load(f)

In [159]:
attention_averages

[tensor([[8.1299e-03, 1.2933e-05, 7.0136e-06, 2.8285e-03, 1.0049e-07, 4.3018e-03,
          3.2891e-03, 2.8185e-04],
         [2.1044e-03, 4.3314e-03, 3.7304e-04, 3.9130e-03, 2.7657e-03, 2.6915e-03,
          3.7362e-03, 2.8121e-03],
         [1.1606e-03, 2.2343e-03, 8.7843e-03, 3.8508e-03, 3.8693e-03, 1.7070e-03,
          3.0324e-03, 3.1653e-04],
         [5.5134e-03, 2.3618e-03, 4.0291e-03, 1.8551e-03, 3.7641e-03, 1.5404e-03,
          4.4523e-03, 3.9748e-03]]),
 tensor([[2.7971e-03, 2.1976e-04, 6.8774e-04, 3.3956e-04, 7.2050e-06, 1.1490e-03,
          2.7872e-03, 4.7563e-04],
         [4.6253e-03, 2.9078e-03, 2.4969e-03, 3.8760e-03, 2.5987e-03, 3.7034e-03,
          3.4816e-03, 2.9449e-03],
         [3.9699e-03, 4.1654e-03, 2.3880e-03, 2.7392e-03, 2.6517e-03, 2.7374e-03,
          3.1106e-03, 4.4681e-03],
         [2.6631e-03, 4.7698e-03, 3.5119e-03, 4.3105e-03, 2.6692e-03, 4.2730e-03,
          3.1826e-03, 3.3612e-03]]),
 tensor([[8.0700e-04, 6.8915e-06, 9.1392e-04, 5.2994e-03, 2.

In [48]:
output_file_path = "/share/home/liangzhongming/930/CGMformer/output/outputs_attentions/shanghai_total_averaged_attention.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(final_merged_attentions, f)